In [ ]:
import os
import csv
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd

In [ ]:
def convert_audio_features(audio_data,sampling_rate):
    spectrogram = librosa.feature.melspectrogram(y=audio_data, sr=sampling_rate)
    mfcc = librosa.feature.mfcc(y=audio_data, sr=sampling_rate)
    chroma = librosa.feature.chroma_stft(y=audio_data, sr=sampling_rate)
    contrast = librosa.feature.spectral_contrast(y=audio_data, sr=sampling_rate)
    tonnetz = librosa.feature.tonnetz(y=audio_data, sr=sampling_rate)

    list = [spectrogram, mfcc, chroma, contrast, tonnetz]
    
    return list

In [ ]:
def plot_audio_features(feature, feature_type):
    plt.figure(figsize=(10, 4))
    
    if feature_type == 'spectrogram':
        librosa.display.specshow(librosa.power_to_db(feature, ref=np.max), x_axis='time', y_axis='mel')
        plt.colorbar(format='%+2.0f dB')
        plt.title('Mel Spectrogram')
        
    elif feature_type == 'mfcc':
        librosa.display.specshow(feature, x_axis='time')
        plt.colorbar()
        plt.title('MFCC')
        
    elif feature_type == 'chroma':
        librosa.display.specshow(feature, x_axis='time', y_axis='chroma')
        plt.colorbar()
        plt.title('Chroma Feature')
        
    elif feature_type == 'contrast':
        librosa.display.specshow(feature, x_axis='time')
        plt.colorbar()
        plt.title('Spectral Contrast')
        
    elif feature_type == 'tonnetz':
        librosa.display.specshow(feature, x_axis='time')
        plt.colorbar()
        plt.title('Tonnetz')
        
    else:
        print("Invalid feature type. Choose from ['spectrogram', 'mfcc', 'chroma', 'contrast', 'tonnetz']")
        return
    
    plt.tight_layout()
    plt.show()

In [ ]:
def plot_all_features(features_list):
    features_dict = {
        'spectrogram': features_list[0],
        'mfcc': features_list[1],
        'chroma': features_list[2],
        'contrast': features_list[3],
        'tonnetz': features_list[4]
    }
    fig, axs = plt.subplots(2, 3, figsize=(15, 10))
    
    for ax, (feature_type, feature) in zip(axs.flatten(), features_dict.items()):
        if feature_type == 'spectrogram':
            im = librosa.display.specshow(librosa.power_to_db(feature, ref=np.max), x_axis='time', y_axis='mel', ax=ax)
            ax.set_title('Mel Spectrogram')
        elif feature_type == 'mfcc':
            im = librosa.display.specshow(feature, x_axis='time', ax=ax)
            ax.set_title('MFCC')
        elif feature_type == 'chroma':
            im = librosa.display.specshow(feature, x_axis='time', y_axis='chroma', ax=ax)
            ax.set_title('Chroma Feature')
        elif feature_type == 'contrast':
            im = librosa.display.specshow(feature, x_axis='time', ax=ax)
            ax.set_title('Spectral Contrast')
        elif feature_type == 'tonnetz':
            im = librosa.display.specshow(feature, x_axis='time', ax=ax)
            ax.set_title('Tonnetz')
        else:
            print(f"Invalid feature type: {feature_type}")
            return

        fig.colorbar(im, ax=ax)

    plt.tight_layout()
    plt.show()

In [ ]:
audio_folder = "C:/Users/victor.serra/OneDrive - americanas s.a/Área de Trabalho/python/chords/variation_chord_audio/"
audio_file_Am7b5_slow = 't_DsGsCsFsAsDs_guitarAsm7b5slowx20201DsGsCsFsAsDs.mp3'
audio_file_Am7b5b_fast = 't_DsGsCsFsAsDs_guitarAsm7b5fastx20201DsGsCsFsAsDs.mp3'
audio_file_Fm7   = 't_DsGsCsFsAsDs_guitarFm7fast002222DsGsCsFsAsDs.mp3'

In [ ]:
Am7b5b_fast = convert_audio_features(librosa.load(f'{audio_folder}/{audio_file_Am7b5b_fast}')[0],22050)
Am7b5b_slow = convert_audio_features(librosa.load(f'{audio_folder}/{audio_file_Am7b5_slow}')[0],22050)
Fm7 = convert_audio_features(librosa.load(f'{audio_folder}/{audio_file_Fm7}')[0],22050)

In [ ]:
Am7b5b_fast[1].shape

In [ ]:
Am7b5b_slow[1].shape

In [ ]:
plot_all_features(Am7b5b_slow)

In [ ]:
plot_all_features(Fm7)

In [ ]:
parsed_files = pd.read_csv('parsed_files.csv')

In [ ]:
parsed_files[parsed_files['file_name'] == 't_DADGBE_banjoD7fast04210.mp3']

In [ ]:
# Pasta onde os arquivos de áudio estão armazenados
audio_folder = "C:/Users/victor.serra/OneDrive - americanas s.a/Área de Trabalho/python/chords/variation_chord_audio/"
csv_file_path = 'audio_features.csv'
parsed_files = pd.read_csv('parsed_files.csv')
parsed_files_list = parsed_files['file_name'].tolist()


# Verificar se o arquivo CSV já existe
file_exists = os.path.isfile(csv_file_path)

# Abrir o arquivo CSV
with open(csv_file_path, 'a', newline='') as csvfile:
    fieldnames = ['file_name', 'chroma', 'contrast', 'tonnetz']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # Escrever o cabeçalho apenas se o arquivo não existir
    if not file_exists:
        writer.writeheader()

    # Percorrer a pasta e processar os arquivos
    for i, filename in enumerate(os.listdir(audio_folder)):
        if filename.endswith('.mp3') and filename not in parsed_files_list:
            print(i, filename)
            if filename.endswith('.mp3'):
                # Limitar o número de arquivos processados para este exemplo
                audio_file_path = os.path.join(audio_folder, filename)
                audio_data, sampling_rate = librosa.load(audio_file_path)

                # Extrair características
                chroma = librosa.feature.chroma_stft(y=audio_data, sr=sampling_rate).tolist()
                contrast = librosa.feature.spectral_contrast(y=audio_data, sr=sampling_rate).tolist()
                tonnetz = librosa.feature.tonnetz(y=audio_data, sr=sampling_rate).tolist()

                # Escrever no CSV
                writer.writerow({
                    'file_name': filename,
                    'chroma': chroma,
                    'contrast': contrast,
                    'tonnetz': tonnetz
                })
            else:
                print(f"Skipping: {filename}")


In [ ]:
# Pasta onde os arquivos de áudio estão armazenados
audio_folder = "C:/Users/victor.serra/OneDrive - americanas s.a/Área de Trabalho/python/chords/variation_chord_audio/"
csv_file_path = 'audio_features_spectrogram.csv'

# Verificar se o arquivo CSV já existe
file_exists = os.path.isfile(csv_file_path)

# Abrir o arquivo CSV
with open(csv_file_path, 'a', newline='') as csvfile:
    fieldnames = ['file_name', 'spectrogram']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # Escrever o cabeçalho apenas se o arquivo não existir
    if not file_exists:
        writer.writeheader()

    # Percorrer a pasta e processar os arquivos
    for i, filename in enumerate(os.listdir(audio_folder)):
        print(i, filename)
        if filename.endswith('.mp3'):
            # Limitar o número de arquivos processados para este exemplo
            audio_file_path = os.path.join(audio_folder, filename)
            audio_data, sampling_rate = librosa.load(audio_file_path)

            # Extrair características
            spectrogram = librosa.feature.melspectrogram(y=audio_data, sr=sampling_rate).tolist()

            # Escrever no CSV
            writer.writerow({
                'file_name': filename,
                'spectrogram': spectrogram
            })

In [ ]:
# Pasta onde os arquivos de áudio estão armazenados
audio_folder = "C:/Users/victor.serra/OneDrive - americanas s.a/Área de Trabalho/python/chords/variation_chord_audio/"
csv_file_path = 'audio_features.csv'

# Verificar se o arquivo CSV já existe
file_exists = os.path.isfile(csv_file_path)

# Abrir o arquivo CSV
with open(csv_file_path, 'a', newline='') as csvfile:
    fieldnames = ['file_name', 'spectrogram', 'mfcc', 'chroma', 'contrast', 'tonnetz']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # Escrever o cabeçalho apenas se o arquivo não existir
    if not file_exists:
        writer.writeheader()

    # Percorrer a pasta e processar os arquivos
    for i, filename in enumerate(os.listdir(audio_folder)):
        if filename.endswith('.mp3'):
            # Limitar o número de arquivos processados para este exemplo
            audio_file_path = os.path.join(audio_folder, filename)
            audio_data, sampling_rate = librosa.load(audio_file_path)

            # Extrair características
            spectrogram = librosa.feature.melspectrogram(y=audio_data, sr=sampling_rate).tolist()
            mfcc = librosa.feature.mfcc(y=audio_data, sr=sampling_rate).tolist()
            chroma = librosa.feature.chroma_stft(y=audio_data, sr=sampling_rate).tolist()
            contrast = librosa.feature.spectral_contrast(y=audio_data, sr=sampling_rate).tolist()
            tonnetz = librosa.feature.tonnetz(y=audio_data, sr=sampling_rate).tolist()

            # Escrever no CSV
            writer.writerow({
                'file_name': filename,
                'spectrogram': spectrogram,
                'mfcc': mfcc,
                'chroma': chroma,
                'contrast': contrast,
                'tonnetz': tonnetz
            })
